# Deno Jupyter Kernel Setup

This notebook uses the Deno Jupyter kernel, which allows running Deno/TypeScript code in Jupyter notebooks.

## Kernel Configuration

The Deno kernel is registered at `~/Library/Jupyter/kernels/deno/kernel.json`. The kernel configuration points to the Deno executable that will be used to run notebook cells.

## Current Setup

- **Deno Version:** Managed via asdf (currently 2.5.6)
- **Kernel Location:** `~/Library/Jupyter/kernels/deno/kernel.json`
- **Kernel Command:** Uses asdf shim to automatically select the active Deno version


5. **Restart kernel in VS Code:**
   - `Cmd+Shift+P` → "Jupyter: Restart Kernel"
   - Or use the restart button in the notebook toolbar

## Verification

Run `Deno.version` in a cell to verify the kernel is using the expected Deno version.


In [1]:
console.dir(Deno.version)

{ deno: "2.5.6", v8: "14.0.365.5-rusty", typescript: "5.9.2" }


In [2]:
console.table({a:1,b:2,c:3})

┌───────┬────────┐
│ (idx) │ Values │
├───────┼────────┤
│ a     │      1 │
│ b     │      2 │
│ c     │      3 │
└───────┴────────┘


In [3]:
import os from "node:os";
import fg from "fast-glob";

/**
 * Finds all books in the specified directory and its subdirectories.
 */
async function findBookPaths(rootPath: string): Promise<string[]> {
  const files = await fg(`${rootPath}/**/*.epub`, {
    concurrency: 1, // just to be kind
    followSymbolicLinks: false, //just to be safe
    // absolute: true, // not necessary,
  });
  // sort files by path because walk/fast-glob returns potentially unsorted files but only under deno?
  files.sort();
  return files;
}
function resolveRootPath(path: string): string {
  const shortcuts: Record<string, string> = {
    test: "./test-books",
    space: "/Volumes/Space/Reading/audiobooks/",
    drop: `${os.homedir()}/Library/CloudStorage/Dropbox/A-Reading/EBook`,
    dropbox: `${os.homedir()}/Library/CloudStorage/Dropbox/A-Reading/EBook`,
  };

  // Resolve shortcut or use the path as is
  const resolvedPath = shortcuts[path] || path;

  // Remove trailing slash and return
  return resolvedPath.replace(/\/$/, "");
}


In [4]:
const rootPath = resolveRootPath("space");
rootPath

"/Volumes/Space/Reading/audiobooks"

In [5]:
const bookPaths = await findBookPaths(rootPath);
console.log(`- Found ${bookPaths.length} books.`);


- Found 517 books.


In [6]:
const bookPath = bookPaths[0]
bookPath


"/Volumes/Space/Reading/audiobooks/Adam Becker - More Everything Forever/Adam Becker - More Everything Forever.epub"

In [7]:
import { parse as parseEpubjs } from "./lib/epubjs-playwright.ts";
import { parse as parseLingo } from "./lib/epub-parser-lingo.ts";
import { compareToc } from "./lib/compare.ts";

const { toc: tocEpubjs } = await parseEpubjs(bookPath, {});
const { toc: tocLingo } = await parseLingo(bookPath, {});
const warnings = compareToc(tocLingo, tocEpubjs, {  });

console.table(warnings)

┌───────┬────────────────┬────────────────────────────────────────────────────────┐
│ (idx) │ type           │ message                                                │
├───────┼────────────────┼────────────────────────────────────────────────────────┤
│     0 │ "toc.presence" │ "lingo produced an empty TOC while epubjs has entries" │
└───────┴────────────────┴────────────────────────────────────────────────────────┘
